In [1]:
%load_ext watermark
%watermark -a "Joel Piper" -d -t -v -p numpy,pandas,nltk,sklearn,gensim -g

Joel Piper 2016-09-19 18:05:12 

CPython 2.7.12
IPython 4.2.0

numpy 1.11.1
pandas 0.18.1
nltk 3.2.1
sklearn 0.17.1
gensim 0.13.2
Git hash: 2e718645ec0e62dd529a4b2784c93c884eff7694


In [3]:
import os
import yaml
import sys

os.chdir('..')

In [4]:
PROJ_ROOT="/Users/Joel/Desktop/Insight/bill_taxonomy/"
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)

In [5]:
%load_ext autoreload
%autoreload 1

In [7]:
%aimport src.ingest.get_bills
%aimport src.wrangle.create_features
%aimport src.analyze.run_model
%aimport src.report.store_db

In [23]:
from src.ingest.get_bills import get_us_bills
from src.ingest.get_bills import get_ny_bills
from src.ingest.get_bills import get_subjects
from src.wrangle.create_features import make_feat_union
from src.analyze.run_model import create_model
from src.analyze.run_model import run_model
from src.wrangle.create_features import make_x_values
from src.wrangle.create_features import make_y_values
from src.analyze.run_model import get_y_probs
from src.report.store_db import store_us_db
from src.report.store_db import store_ny_db

with open("configs.yml", 'r') as ymlfile:
        cfg = yaml.load(ymlfile)

        dbname = cfg['dbname']
        username = cfg['username']
        us_bills_subset = cfg['us_bills_subset']
        subject_list = cfg['subjects']
        pipe_feats = cfg['pipe_feats']
        model_type = cfg['model_type']
        us_bills = get_us_bills(dbname, username, us_bills_subset)
        subjects = get_subjects(dbname, username, subject_list)
        X = make_x_values(us_bills)
        feat_un, feat_params = make_feat_union(pipe_feats, cfg)
        model = create_model(feat_un, model_type, feat_params, cfg)

        results = []
        for sub in subject_list:
            y = make_y_values(us_bills, subjects, sub)
            fit_mod = run_model(model, X, y, sub, cfg)
            results.append(fit_mod)

            if(cfg['store_us']):
                y_probs_us = get_y_probs(fit_mod, X)
                store_us_db(dbname, username, us_bills, sub, y_probs_us, y, cfg)

            if(cfg['store_ny']):
                ny_bills_subset = cfg['ny_bills_subset']
                ny_bills = get_ny_bills(dbname, username, ny_bills_subset)
                X_ny = make_x_values(ny_bills)
                y_probs_ny = get_y_probs(fit_mod, X_ny)
                store_ny_db(dbname, username, ny_bills, sub, y_probs_ny, cfg)


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] features__lda_text__lda_tf_text__min_df=10, model__C=10, features__tfidf_title__max_df=0.4, features__tfidf_title__ngram_range=(1, 3), features__tfidf_title__min_df=10, features__tfidf_text__max_features=None, features__tfidf_text__ngram_range=(1, 2), model__penalty=l1, features__lda_text__lda_model_text__n_topics=100, features__lda_text__lda_tf_text__ngram_range=(1, 1), features__tfidf_title__max_features=None, features__lda_text__lda_tf_text__max_df=0.4, features__tfidf_text__min_df=10, features__tfidf_text__max_df=0.4 


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 24: ordinal not in range(128)

In [ ]:
%debug

> /Users/Joel/anaconda/envs/insight/lib/python2.7/site-packages/nltk/corpus/reader/wordnet.py(1694)apply_rules()
   1692                     for form in forms
   1693                     for old, new in substitutions
-> 1694                     if form.endswith(old)]
   1695 
   1696         def filter_forms(forms):

ipdb> u
> /Users/Joel/anaconda/envs/insight/lib/python2.7/site-packages/nltk/corpus/reader/wordnet.py(1712)_morphy()
   1710 
   1711         # 1. Apply rules once to the input to get y1, y2, y3, etc.
-> 1712         forms = apply_rules([form])
   1713 
   1714         # 2. Return all that are in the database (and check the original too)

ipdb> u
> /Users/Joel/anaconda/envs/insight/lib/python2.7/site-packages/nltk/stem/wordnet.py(40)lemmatize()
     38 
     39     def lemmatize(self, word, pos=NOUN):
---> 40         lemmas = wordnet._morphy(word, pos)
     41         return min(lemmas, key=len) if lemmas else word
     42 

ipdb> u
> /Users/Joel/Desktop/Insigh

In [ ]:
    import psycopg2
    import pandas as pd
    con = psycopg2.connect(database='bills_db', user='Joel')
    ny_subset = 10
    # query:
    sql_str = """
    SELECT bill_num, bill_name, bill_text FROM ny_bills
    LIMIT {0}
    """
    sql_query = sql_str.format(ny_subset)
    ny_bills = pd.read_sql_query(sql_query, con)

In [ ]:
ny_bills.ix[0,'bill_name'].lower()

In [ ]:
bill_cols = ny_bills[['bill_name', 'bill_text']]
test_tuple = [tuple(x) for x in bill_cols.values]

In [ ]:
test_tuple[0][0].lower()

In [ ]:
us_bill_cols = us_bills[['bill_name', 'bill_text']]
us_test_tuple = [tuple(x) for x in us_bill_cols.values]

In [ ]:
us_test_tuple[0][0]